In [ ]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms, models
from torch.utils.data import Dataset, DataLoader
from PIL import Image

class AgeDataset(Dataset):
    def __init__(self, data_path, annot_path, train=True, transform=None):
        self.annot_df = pd.read_csv(annot_path)
        self.data_path = data_path
        self.train = train
        self.transform = transform

    def __len__(self):
        return len(self.annot_df)

    def __getitem__(self, idx):
        img_name = self.annot_df.iloc[idx, 0]
        img_path = self.data_path + '/' + img_name
        image = Image.open(img_path).convert('RGB')
        
        if self.transform:
            image = self.transform(image)

        if self.train:
            age = self.annot_df.iloc[idx, 1]
            return image, age
        else:
            return image


train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])



train_path = '/kaggle/input/smai-24-age-prediction/content/faces_dataset/train'
train_ann = '/kaggle/input/smai-24-age-prediction/content/faces_dataset/train.csv'


test_path = '/kaggle/input/smai-24-age-prediction/content/faces_dataset/test'
test_ann = '/kaggle/input/smai-24-age-prediction/content/faces_dataset/submission.csv'


train_dataset = AgeDataset(train_path, train_ann, train=True, transform=train_transform)
test_dataset = AgeDataset(test_path, test_ann, train=False, transform=train_transform)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)


class AgePredictionModel(nn.Module):
    def __init__(self, model_type='resnet18'):
        super(AgePredictionModel, self).__init__()
        if model_type == 'resnet18':
            self.model = models.resnet18(pretrained=True)
        elif model_type == 'resnet50':
            self.model = models.resnet50(pretrained=True)
        elif model_type == 'resnet101':
            self.model = models.resnet101(pretrained=True)
        else:
            raise ValueError("Invalid model type")
            
        num_ftrs = self.model.fc.in_features
        self.model.fc = nn.Linear(num_ftrs, 1)

    def forward(self, x):
        return self.model(x)


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
models_to_ensemble = ['resnet18', 'resnet50', 'resnet101']
predictions_ensemble = []

for model_type in models_to_ensemble:
    model = AgePredictionModel(model_type).to(device)
    
   
    for param in model.model.parameters():
        param.requires_grad = False
    for param in model.model.layer4.parameters():
        param.requires_grad = True
    
    criterion = nn.SmoothL1Loss()
    optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=0.001)
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

    num_epochs = 20
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for i, data in enumerate(train_loader, 0):
            inputs, labels = data[0].to(device), data[1].float().unsqueeze(1).to(device)
            optimizer.zero_grad()

            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            if i % 10 == 9:
                print('[%s, %d, %5d] loss: %.3f' % (model_type, epoch + 1, i + 1, running_loss / 10))
                running_loss = 0.0
        scheduler.step()

    print('Finished Training', model_type)



    model.eval()
    predictions = []
    with torch.no_grad():
        for data in test_loader:
            inputs = data.to(device)
            outputs = model(inputs)
            predictions.extend(outputs.cpu().numpy().flatten())
    
    predictions_ensemble.append(predictions)


ensemble_predictions = np.mean(predictions_ensemble, axis=0)

ensemble_predictions = np.round(ensemble_predictions).astype(int)

test_ann_df = pd.read_csv(test_ann)
test_ann_df['age'] = ensemble_predictions
test_ann_df.to_csv('final_output.csv', index=False)
print("DONE")